In [1]:
import csv
import numpy as np #We will make extensive use of Numpy arrays 
#import matplotlib  #ploting
#matplotlib.use("Agg")  #some hack to stop the bouncing python icon when plotting of os/x
#import matplotlib.pyplot as plt
#from matplotlib.colors import LogNorm #for better display of FITS images
#%matplotlib inline
from astropy.time import Time

In [2]:
toi_file='toi-2019-05-07.csv'

In [3]:
tzero=2457000

In [4]:
tidid=[]
toiid=[]
ra=[]
dec=[]
tmag=[]
t0=[]
t0err=[]
per=[]
pererr=[]
tdur=[]
tdurerr=[]
tdep=[]
tdeperr=[]
with open(toi_file, newline='') as csvfile:
    csvdata = csv.reader(csvfile, delimiter=',')
    i=0
    for row in csvdata:
        i=i+1
        if i > 1 and row[10]!='' and row[11]!='' and row[15]!='':
            tidid.append(float(row[1]))
            toiid.append(float(row[2]))
            ra.append(float(row[4]))
            dec.append(float(row[5]))
            tmag.append(float(row[6]))
            t0.append(float(row[8])+2457000.0)
            t0err.append(float(row[9]))
            per.append(float(row[10]))
            pererr.append(float(row[11]))
            tdur.append(float(row[12]))
            tdurerr.append(float(row[13]))
            tdep.append(float(row[14]))
            tdeperr.append(float(row[15]))

In [5]:
t = Time(t0, format='jd', scale='tcb')

In [6]:
tnow = Time(['2019-05-01 00:00:00','2019-08-30 00:00:00'], format='iso', scale='utc')

In [7]:
print(t.utc.iso[1])
print(tnow.jd)

2019-02-28 17:25:50.288
[2458604.5 2458725.5]


In [8]:
f=open('TESS_20190507.csv','w')
file=csv.writer(f,delimiter='|')
file.writerow(['TOI','Mid Time','Start Time','End Time','Ingress Start','Ingress End'\
               ,'Egress Start','Egress End'])
nstar=len(t0)
sortidx=np.argsort(tmag)
#for i in range(nstar):
for i in sortidx[:]:
    if tdep[i] > 3000.0 and tdep[i] < 20000.0 and tmag[i] < 10 and toiid[i] != 129.01:
        #file.writerow(['TOI',toiid[i],ra[i],dec[i],tmag[i],tdep[i]])
        n_start=int((tnow.jd[0]-t.jd[i])/per[i]+1)
        n_finish=int((tnow.jd[1]-t.jd[i])/per[i]+1)
        tmid=[]
        tstart=[]
        tend=[]
        ting_start=[]
        ting_end=[]
        teg_start=[]
        teg_end=[]
        for j in range(n_start,n_finish):
            tmid.append(t0[i]+j*per[i])
            tstart.append(t0[i]+j*per[i]-tdur[i]/24.0/1.0-t0err[i])
            tend.append(t0[i]+j*per[i]+tdur[i]/24.0/1.0+t0err[i])
            ting_start.append(t0[i]+j*per[i]-tdur[i]/24.0/2.0-t0err[i])
            ting_end.append(t0[i]+j*per[i]-tdur[i]/24.0/2.0+t0err[i])
            teg_start.append(t0[i]+j*per[i]+tdur[i]/24.0/2.0-t0err[i])
            teg_end.append(t0[i]+j*per[i]+tdur[i]/24.0/2.0+t0err[i])
        tmid_times=Time(tmid, format='jd', scale='tcb')
        tstart_times=Time(tstart, format='jd', scale='tcb')
        tend_times=Time(tend, format='jd', scale='tcb')
        ting_start_times=Time(ting_start, format='jd', scale='tcb')
        ting_end_times=Time(ting_end, format='jd', scale='tcb')
        teg_start_times=Time(teg_start, format='jd', scale='tcb')
        teg_end_times=Time(teg_end, format='jd', scale='tcb')
        for j in range(len(tmid_times.iso)):
            #print(tstart_times.iso[j],'|',tmid_times.iso[j],'|',tend_times.iso[j],\
            #      '|',ting_start_times.iso[j],'|',ting_end_times.iso[j],\
            #      '|',teg_start_times.iso[j],'|',teg_end_times.iso[j])
            #file.writerow(['TOI'+str(toiid[i]),tstart_times.utc.iso[j],ting_end_times.utc.iso[j],tmid_times.utc.iso[j],\
            #      teg_start_times.utc.iso[j],tend_times.utc.iso[j]])
            file.writerow(['TOI'+str(toiid[i]),tmid_times.utc.iso[j],tstart_times.utc.iso[j],\
                           tend_times.utc.iso[j],ting_start_times.utc.iso[j],ting_end_times.utc.iso[j],\
                           teg_start_times.utc.iso[j],teg_end_times.utc.iso[j]])
        file.writerow(' ')
f.close()

In [9]:
sortidx=np.argsort(toiid)
for i in sortidx[:]:
    if tdep[i] > 3000.0 and tdep[i] < 20000.0 and tmag[i] < 10 and toiid[i] != 129.01:
        print('TOI'+str(toiid[i]),',',ra[i],',',dec[i],',',tmag[i],',',tdur[i],',',tdep[i],',',t0[i],',',per[i])

TOI102.01 , 87.139956 , -63.988427 , 9.716 , 3.77916 , 14034.13547 , 2458326.078557 , 4.411955
TOI104.01 , 319.949611 , -58.148876 , 9.848 , 5.59014 , 3586.196984 , 2458327.673462 , 4.087445
TOI105.01 , 337.457199 , -48.003087 , 9.48 , 2.869426 , 11836.69122 , 2458326.50599 , 2.184659
TOI107.01 , 313.783108 , -34.135572 , 9.639 , 4.557506 , 12999.89718 , 2458328.29934 , 3.950054
TOI121.01 , 331.867232 , -41.815456 , 9.929 , 5.449073 , 14107.16966 , 2458336.139719 , 14.775056
TOI123.01 , 319.6994 , -26.616077 , 8.803 , 5.634728 , 3177.131896 , 2458325.375658 , 3.308754
TOI131.01 , 353.826767 , -64.561769 , 9.737 , 1.569129 , 9113.476719 , 2458326.068952 , 6.240662
TOI135.01 , 7.328937 , -76.304075 , 9.213 , 4.492686 , 10068.55437 , 2458325.783759 , 4.126904
TOI138.01 , 328.926194 , -61.673286 , 9.534 , 2.176782 , 3722.005937 , 2458328.058635 , 6.198041
TOI143.01 , 328.767654 , -22.612566 , 9.772 , 3.422324 , 6939.353936 , 2458325.58249 , 2.310894
TOI155.01 , 321.114974 , -37.850144 , 9.

In [10]:
sortidx=np.argsort(toiid)
for i in sortidx[:]:
    if tdep[i] > 3000.0 and tdep[i] < 20000.0 and tmag[i] < 10 and toiid[i] != 129.01:
        print('TOI'+str(toiid[i]),',',t0[i],',',per[i],',',tmag[i],',',tdur[i],',',tdep[i])

TOI102.01 , 2458326.078557 , 4.411955 , 9.716 , 3.77916 , 14034.13547
TOI104.01 , 2458327.673462 , 4.087445 , 9.848 , 5.59014 , 3586.196984
TOI105.01 , 2458326.50599 , 2.184659 , 9.48 , 2.869426 , 11836.69122
TOI107.01 , 2458328.29934 , 3.950054 , 9.639 , 4.557506 , 12999.89718
TOI121.01 , 2458336.139719 , 14.775056 , 9.929 , 5.449073 , 14107.16966
TOI123.01 , 2458325.375658 , 3.308754 , 8.803 , 5.634728 , 3177.131896
TOI131.01 , 2458326.068952 , 6.240662 , 9.737 , 1.569129 , 9113.476719
TOI135.01 , 2458325.783759 , 4.126904 , 9.213 , 4.492686 , 10068.55437
TOI138.01 , 2458328.058635 , 6.198041 , 9.534 , 2.176782 , 3722.005937
TOI143.01 , 2458325.58249 , 2.310894 , 9.772 , 3.422324 , 6939.353936
TOI155.01 , 2458326.74444 , 5.43636 , 9.0 , 5.974 , 8232.2
TOI165.01 , 2458388.296648 , 7.76178 , 9.789 , 3.494972 , 4399.776458
TOI182.01 , 2458355.082723 , 1.769141 , 9.623 , 1.958463 , 9408.639864
TOI183.01 , 2458326.104287 , 3.430776 , 8.273 , 3.130096 , 8980.21782
TOI185.01 , 2458354.45801